In [126]:
import pandas as pd
import numpy as np


In [127]:
df = pd.read_csv("spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [128]:
df.Category.value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [129]:
df['spam'] = df['Category'].apply(lambda x: 1 if x =='spam' else 0)  # spam =1 , ham = 0

In [130]:
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [131]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Message, df.spam, test_size=0.2)


In [132]:
X_train.shape

(4457,)

In [133]:
X_test.shape

(1115,)

In [134]:
type(X_train)

pandas.core.series.Series

In [135]:
X_train[:4]

3335    That's fine, have him give me a call if he kno...
3044           Your bill at 3 is £33.65 so thats not bad!
2351                 You will be in the place of that man
485     You should change your fb to jaykwon thuglyfe ...
Name: Message, dtype: object

In [136]:
type(y_train)

pandas.core.series.Series

In [137]:
y_train[:4]

3335    0
3044    0
2351    0
485     0
Name: spam, dtype: int64

In [138]:
X_train.values

array(["That's fine, have him give me a call if he knows what he wants or has any questions",
       'Your bill at 3 is £33.65 so thats not bad!',
       'You will be in the place of that man', ...,
       'Yeh. Indians was nice. Tho it did kane me off a bit he he. We shud go out 4 a drink sometime soon. Mite hav 2 go 2 da works 4 a laugh soon. Love Pete x x',
       'Or just do that 6times', "Sorry, I'll call later"], dtype=object)

In [139]:
type(X_train.values)

numpy.ndarray

Create bag of words representation using CountVectorizer

In [141]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(X_train.values)
X_train_cv

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 59457 stored elements and shape (4457, 7785)>

In [142]:
X_train_cv.toarray()[:2][0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [143]:
X_train_cv.shape

(4457, 7785)

In [144]:
v.get_feature_names_out()[1000:1050]

array(['antibiotic', 'any', 'anybody', 'anyhow', 'anymore', 'anyone',
       'anyones', 'anyplaces', 'anythiing', 'anythin', 'anything',
       'anythingtomorrow', 'anytime', 'anyway', 'anyways', 'aom', 'apart',
       'apartment', 'apes', 'apeshit', 'aphex', 'apnt', 'apo',
       'apologetic', 'apologise', 'apologize', 'apology', 'app',
       'apparently', 'appeal', 'appear', 'appendix', 'applausestore',
       'applebees', 'apples', 'apply', 'applyed', 'appointment',
       'appreciate', 'appreciated', 'approaching', 'appropriate',
       'approve', 'approved', 'approx', 'apps', 'appt', 'appy', 'april',
       'aproach'], dtype=object)

In [145]:
v.get_feature_names_out().shape

(7785,)

v.vocabulary_

In [147]:
v.get_feature_names_out()[6543]

'stops'

In [148]:
X_train_np = X_train_cv.toarray()
X_train_np[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [149]:
np.where(X_train_np[0]!=0)   # it will value 1

(array([1001, 1616, 2882, 3169, 3376, 3389, 3399, 3467, 3633, 4001, 4437,
        5005, 5566, 6853, 7421, 7517], dtype=int64),)

In [153]:
X_train[:4]

3335    That's fine, have him give me a call if he kno...
3044           Your bill at 3 is £33.65 so thats not bad!
2351                 You will be in the place of that man
485     You should change your fb to jaykwon thuglyfe ...
Name: Message, dtype: object

In [155]:
X_train_np[0][5677]

0

In [157]:
v.get_feature_names_out()[5677]

'recorded'

Now we will build machine learning model

In [160]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB()

In [162]:
X_train_cv = v.transform(X_test)

In [164]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_train_cv)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.95      0.91      0.93       150

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [166]:

emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]
emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1], dtype=int64)

In [168]:

from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [170]:
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [171]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.95      0.91      0.93       150

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115

